In [1]:
import numpy as np
import pandas as pd

In [2]:
data_paths = {
    2011: "https://www.mlit.go.jp/common/000184224.xls",
    2012: "https://www.mlit.go.jp/common/000994340.xls",
    2013: "https://www.mlit.go.jp/common/001032144.xls",
    2014: "https://www.mlit.go.jp/common/001084270.xls",
    2015: "https://www.mlit.go.jp/common/001173128.xls",
    2016: "https://www.mlit.go.jp/common/001179510.xls",
    2017: "https://www.mlit.go.jp/common/001226298.xls",
    2018: "https://www.mlit.go.jp/common/001283010.xls",
    2019: "https://www.mlit.go.jp/kankocho/siryou/toukei/content/001335744.xls",
}

In [3]:
fix_countries = {"中国（台湾）": "台湾", "中国 [香港]": "香港"}
fix_port = {
    "新千歳": "新千歳空港",
    "函館空港": "函館空港",
    "仙台": "仙台空港",
    "新潟": "新潟空港",
    "羽田": "東京国際空港（羽田空港）",
    "成田": "成田国際空港",
    "小松": "小松空港",
    "富士山静岡": "富士山静岡空港",
    "中部": "中部国際空港",
    "関西": "関西国際空港",
    "広島": "広島空港",
    "関門（下関）港": "関門（下関）港",
    "高松": "高松空港",
    "福岡": "福岡空港",
    "博多港": "博多港",
    "厳原港": "厳原港",
    "鹿児島": "鹿児島空港",
    "那覇": "那覇空港",
}

In [4]:
airport_lat = {
    "新千歳空港": 42.775,
    "函館空港": 41.77,
    "仙台空港": 38.136944,
    "新潟空港": 37.955833,
    "東京国際空港（羽田空港）": 35.553333,
    "成田国際空港": 35.765278,
    "小松空港": 36.393889,
    "富士山静岡空港": 34.796111,
    "中部国際空港": 34.858333,
    "関西国際空港": 34.434167,
    "広島空港": 34.436111,
    "関門（下関）港": 33.957889,
    "高松空港": 34.214167,
    "福岡空港": 33.584444,
    "博多港": 33.605133,
    "厳原港": 34.1977845,
    "鹿児島空港": 31.8,
    "那覇空港": 26.205556,
}

airport_lon = {
    "新千歳空港": 141.692222,
    "函館空港": 140.821944,
    "仙台空港": 140.9225,
    "新潟空港": 139.120556,
    "東京国際空港（羽田空港）": 139.781111,
    "成田国際空港": 140.385556,
    "小松空港": 136.4075,
    "富士山静岡空港": 138.189444,
    "中部国際空港": 136.805278,
    "関西国際空港": 135.232778,
    "広島空港": 132.919444,
    "関門（下関）港": 130.941333,
    "高松空港": 134.015556,
    "福岡空港": 130.451667,
    "博多港": 130.397414,
    "厳原港": 129.2913508,
    "鹿児島空港": 130.721667,
    "那覇空港": 127.650833,
}

In [5]:
countries = {
    "韓国": "KOR",
    "台湾": "TWN",
    "香港": "HKG",
    "中国": "CHN",
    "タイ": "THA",
    "シンガポール": "SGP",
    "マレーシア": "MYS",
    "インドネシア": "IDN",
    "フィリピン": "PHL",
    "ベトナム": "VNM",
    "インド": "IND",
    "英国": "GBR",
    "ドイツ": "DEU",
    "フランス": "FRA",
    "イタリア": "ITA",
    "スペイン": "ESP",
    "ロシア": "RUS",
    "米国": "USA",
    "カナダ": "CAN",
    "オーストラリア": "AUS",
}

In [6]:
def get_entry_data(year):
    if year > 2017:
        sheet_name = "表1-1"
    else:
        sheet_name = "第１表"
    df = pd.read_excel(data_paths[year], sheet_name=sheet_name, skiprows=4,)

    entry_df = (
        df.loc[2:, ~df.iloc[0, :].str.startswith("構成比").fillna(False)]
        .iloc[:, 1:]
        .copy()
    )
    entry_df = (
        entry_df.set_index("調査項目")
        .loc["入国空港・海港":"出国空港・海港"]
        .iloc[:-1, :]
        .drop(["Unnamed: 3"], axis=1)
        .rename({"Unnamed: 2": "空港・海港"}, axis=1)
    )
    for col in entry_df.columns[1:]:
        entry_df[col] = entry_df[col].astype(float)

    entry_tidy_df = entry_df.melt(
        id_vars=["空港・海港"],
        value_vars=entry_df.columns[1:],
        var_name="国籍・地域",
        value_name="入国",
    )
    entry_tidy_df["国籍・地域"] = entry_tidy_df["国籍・地域"].apply(
        lambda x: fix_countries.get(x, x)
    )
    entry_tidy_df["空港・海港"] = entry_tidy_df["空港・海港"].apply(lambda x: fix_port.get(x, x))
    entry_tidy_df["alpha-3"] = entry_tidy_df["国籍・地域"].apply(
        lambda x: countries.get(x, np.nan)
    )
    entry_tidy_df["lat"] = entry_tidy_df["空港・海港"].apply(
        lambda x: airport_lat.get(x, np.nan)
    )
    entry_tidy_df["lon"] = entry_tidy_df["空港・海港"].apply(
        lambda x: airport_lon.get(x, np.nan)
    )
    entry_tidy_df["年度"] = year
    #     entry_tidy_df.dropna(inplace=True)
    return entry_tidy_df

In [7]:
df = pd.concat([get_entry_data(year) for year in sorted(data_paths.keys())])

In [8]:
df.to_csv("entered_port.csv", index=False)